In [1]:
import pandas as pd
import requests
import time
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
from datetime import datetime
from typing import List

In [3]:
YEAR_IN_SECONDS = 31536000
EXPIRY = 'expiration_timestamp_ms'
INSTRUMENT_TYPE = 'instrument_type'
INSTRUMENT_NAME = 'instrument_name'
CCY = 'base_ccy'
IS_OPEN = 'is_open'
FUT_TYPE = 'settlement_period'
MARK_PX = 'mark_price'
USE_PX = 'use_price'
DISCARD_THRESHOLD_FACTOR = 1.5
FUTURES_DATA_COLS = [INSTRUMENT_NAME, CCY, EXPIRY, MARK_PX, USE_PX]
OPT_DATA_COLS = [INSTRUMENT_NAME, CCY, EXPIRY, 'spot']
url = "https://api-internal.prod.blockchain.info/altonomy/ple-deribit-pricer"
binance_sapi = "https://api.binance.com/api/v3/ticker/bookTicker"
binance_dapi = "https://dapi.binance.com/dapi/v1"
binance_fapi = "https://fapi.binance.com/fapi/v1"
OKX_API = "https://www.okx.com/api/v5"
OKX_TICKER_ENDPOINT = "/market/ticker"
OKX_TICKERS_ENDPOINT = "/market/tickers"
OKX_INSTRUMENTS_ENDPOINT = "/public/instruments"
OKX_FUNDING_ENDPOINT = "/public/funding-rate"
deri_api = "https://www.deribit.com/api/v2/public"
kraken_api = "https://api.kraken.com/0/public/Ticker"
KRAKEN_BASE_API = "https://futures.kraken.com/derivatives/api/v3"

In [9]:
def to_df(response):
    df=pd.json_normalize(response.json(),max_level=0).set_index('instrument_id')
    for col in ['exchange_ms', 'interval_start_ms', 'interval_end_ms' ]:
        if col in df.columns:
            df[col]=pd.to_datetime(df[col], unit='ms')
    for col in ['update_ts']:
        if col in df.columns:
            df[col]=pd.to_datetime(df[col], unit='us')
    return df
def get_all_vol():
    dfs=[]
    for cat in [ "symbol", "bbo", "ticker", "trade", "volprice"]:
        response = requests.get(url + "/" + cat)
        if response and response.ok :
            print("got ", cat)
            # Print the response
            dfs.append(to_df(response))
    return pd.concat(dfs, axis=1)
def get_spot_prices():
    #  This function returns mids for: BTCUSDT, ETHUSDT and USDTUSD
    response = requests.get(binance_sapi, params = {'symbol' : 'BTCUSDT'})
    json_res = response.json()
    mid_btc_px = (float(json_res['bidPrice']) + float(json_res['askPrice']))/2
    response = requests.get(binance_sapi, params = {'symbol' : 'ETHUSDT'})
    json_res = response.json()
    mid_eth_px = (float(json_res['bidPrice']) + float(json_res['askPrice']))/2
    response = requests.get(kraken_api + "?pair=USDTUSD")
    json_res = response.json()['result']['USDTZUSD']
    mid_usdt_px = (float(json_res['a'][0]) + float(json_res['b'][0]))/2
    return (mid_btc_px, mid_eth_px, mid_usdt_px)
def get_deribit_funding():
    response = requests.get(deri_api + "/get_time?")
    json_res = response.json()
    server_time = json_res['result']
    response = requests.get(deri_api + f"/get_funding_rate_value?instrument_name=BTC-PERPETUAL&start_timestamp={server_time - 8 * 3688 * 1000}&end_timestamp={server_time}")
    json_res = response.json()
    btc_rate = json_res['result']
    response = requests.get(deri_api + f"/get_funding_rate_value?instrument_name=ETH-PERPETUAL&start_timestamp={server_time - 8 * 3688 * 1000}&end_timestamp={server_time}")
    json_res = response.json()
    eth_rate = json_res['result']
    return (btc_rate, eth_rate)
def preprocess_futures_data(input_df: pd.DataFrame, btc_spot: float, eth_spot: float):
    # btc_spot and eth_spot aguments expected here are BTCUSD and ETHUSD!!!
    ## save the futures data separately
    input_df_futures = input_df[(input_df[INSTRUMENT_TYPE] == 'Future') & (input_df[IS_OPEN]==True)].copy()
    input_df_futures[USE_PX] = input_df_futures[MARK_PX]
    ## set the perp maturity to always be now + 8hrs
    input_df_futures.loc[input_df_futures['settlement_period'] == 'Perpetual', EXPIRY] = int(time.time() * 1000 + 8 * 3600 * 1000)
    (btc_funding, eth_funding) = get_deribit_funding()
    # add funding payment to spot to get the adjusted perp price
    # starting point: buy 1 unit of spot for SPOT_PX, sell SPOT_PX USD worth of perp, hold for 8hrs
    # funding payment in coin = Funding Rate * POSITION_SIZE_COIN => funding payment in USD = Funding Rate * SPOT_PX
    input_df_futures.loc[(input_df_futures['settlement_period'] == 'Perpetual') & (input_df_futures[CCY] == 'BTC'), USE_PX] = btc_spot * (1 + btc_funding)
    input_df_futures.loc[(input_df_futures['settlement_period'] == 'Perpetual') & (input_df_futures[CCY] == 'ETH'), USE_PX] = eth_spot * (1 + eth_funding)
    ## extract only the necessary fields
    df_futures_btc = input_df_futures.filter(FUTURES_DATA_COLS).loc[(input_df_futures[CCY] == 'BTC')]
    df_futures_eth = input_df_futures.filter(FUTURES_DATA_COLS).loc[(input_df_futures[CCY] == 'ETH')]
    df_futures_btc.loc[-1] = ['BTC-SPOT', 'BTC', time.time() * 1000, btc_spot, btc_spot]
    df_futures_eth.loc[-1] = ['ETH-SPOT', 'ETH', time.time() * 1000, eth_spot, eth_spot]
    df_futures_btc.sort_values(by = EXPIRY, inplace = True)
    df_futures_eth.sort_values(by = EXPIRY, inplace = True)
    df_futures_btc.reset_index(drop = True, inplace = True)
    df_futures_eth.reset_index(drop = True, inplace = True)
    return df_futures_btc, df_futures_eth
def get_binance_funding(base_url: str, underlying_pair: str):
    ticker_pi = base_url + "/premiumIndex?symbol="
    response = requests.get(ticker_pi + underlying_pair)
    f_rate = response.json()
    # list of dicts is returned by dapi, single dict by fapi
    if isinstance(f_rate, list):
        f_rate = f_rate[0]
    latest_rate = float(f_rate['lastFundingRate'])
    return latest_rate
def get_binance_futures(base_url: str, underlying_pair: str, spot_px: float, is_usdm: bool = True):
    # if this is a coin-m futures, spot_px has to be /USD and multiplier has to be set to 1
    # Otherwise, spot_px is /USDT and multiplier has to be set to USDT/USD
    exch_info = base_url + "/exchangeInfo"
    ticker_ob = base_url + "/ticker/bookTicker?symbol="
    response = requests.get(exch_info)
    allsym = pd.DataFrame(response.json()['symbols'])
    input_futs = allsym.loc[allsym['pair'] == underlying_pair]
    futs = []
    for index, row in input_futs.iterrows():
        fut = {}
        ticker_mkt = requests.get(ticker_ob + row['symbol']).json()
        # list of dicts is returned by dapi, single dict by fapi
        if isinstance(ticker_mkt, list):
            ticker_mkt = ticker_mkt[0]
        fut[MARK_PX] = (float(ticker_mkt['bidPrice']) + float(ticker_mkt['askPrice'])) / 2
        fut[USE_PX] = fut[MARK_PX]
        fut[INSTRUMENT_NAME] = row['symbol']
        fut[CCY] = underlying_pair
        if row['contractType'] == "PERPETUAL":
            f_rate = get_binance_funding(base_url, row['symbol'])
            # add funding payment to spot to get the adjusted perp price
            if is_usdm:
                # starting point: buy 1 unit of spot for SPOT_PX, sell 1 unit of perp, hold for 8hrs
                # funding payment in USD = Funding Rate * size of 1 contract in USD => 1 contract * MARK_PX * Funding rate
                fut[USE_PX] = spot_px + fut[MARK_PX] * f_rate
            else:
                # starting point: buy 1 unit of spot for SPOT_PX, sell SPOT_PX USD worth of perp, hold for 8hrs
                # funding payment in coin = Funding Rate * POSITION_SIZE_USD => funding payment in USD = Funding Rate * POSITION_SIZE_USD/MARK_PX * SPOT_PX
                fut[USE_PX] = spot_px * (1 + spot_px / fut[MARK_PX] * f_rate)
            fut[EXPIRY] = int(time.time() * 1000 + 8 * 3600 * 1000)
        else:
            fut[EXPIRY] = int(row['deliveryDate'])
        futs.append(fut)
    bin_futs = pd.DataFrame(futs, columns = FUTURES_DATA_COLS)
    bin_futs.loc[-1] = [underlying_pair, underlying_pair, time.time() * 1000, spot_px, spot_px]
    bin_futs.sort_values(by = EXPIRY, inplace = True)
    bin_futs.reset_index(drop = True, inplace = True)
    return bin_futs
def get_okx_ticker(inst_id: str) -> dict:
    """
    @notice return market data for instrument id, ie OHLC / current price / volume
    @param inst_id, instrument id for coin, i.e. 'BTC-USD-SWAP' for btc perp
    """
    url = OKX_API + OKX_TICKER_ENDPOINT
    params = {"instId": inst_id.upper()}
    resp = requests.get(url=url, params=params)
    data = resp.json()
    return data
def get_okx_tickers(coin: str, inst_type: str = "FUTURES") -> List[str]:
    """
    @notice return instrument names of all inst_types associated with coin
    @param coin, name of coin to return instruments for
    """
    url = OKX_API + OKX_TICKERS_ENDPOINT
    resp = requests.get(url, params={"instType": inst_type.upper()})
    data = resp.json()
    return [d['instId'] for d in data['data'] if d['instId'].startswith(f'{coin.upper()}-')]
def get_okx_instruments(coin: str, instFamily: str) -> List[dict]:
    """
    @notice returns instrument info - including expiration date for futures,
            which is not included in the /tickers endpoint
    """
    resp = requests.get(OKX_API + OKX_INSTRUMENTS_ENDPOINT, {'instType': 'FUTURES'})
    data = resp.json()['data']
    return [d for d in data if d['instFamily'] == instFamily]
def get_okx_funding(perp: str) -> float:
    resp = requests.get(OKX_API + OKX_FUNDING_ENDPOINT, {'instId': perp})
    data = resp.json()['data'][0]
    return float(data['fundingRate'])
def sort_okx_instruments(instruments: list) -> List[str]:
    """
    @notice this sorts instruments based on string date, i.e.,
        ['BTC-USDT-240426',            ['BTC-USDT-240329',
        'BTC-USDT-240329']    --->     'BTC-USDT-240426']
    """
    term_instruments_ = list(instruments)
    term_split = [i.split('-') for i in term_instruments_]
    term_sorted = [(base, quote, datetime.strptime(date, '%y%m%d')) for base, quote, date in term_split]
    term_sorted.sort(key=lambda x: x[-1])
    return [f"{base}-{quote}-{datetime.strftime(date, '%y%m%d')}" for base, quote, date in term_sorted]


def get_okx_futures(coin: str, baseCcy: str, spot_px: float, is_usdm: bool = True):
    term_instruments = sort_okx_instruments(get_okx_tickers(f"{coin.upper()}-{baseCcy.upper()}"))
    perp_instrument = f"{coin.upper()}-{baseCcy.upper()}-SWAP" # /USD or /USDT perps available
    spot = {}
    spot[EXPIRY] = time.time() * 1000
    spot[MARK_PX] = spot_px
    spot[INSTRUMENT_NAME] = f"{coin.upper()}-{baseCcy.upper()}-SPOT"
    spot[CCY] = f"{coin.upper()}-{baseCcy.upper()}"
    spot[USE_PX] = spot[MARK_PX]
    all_inst = [spot]
    # set perp expiry
    perp = {}
    perp[EXPIRY] = int(time.time() * 1000 + 8 * 3600 * 1000)
    # deal with the perp first
    perp_data = get_okx_ticker(perp_instrument)['data'][0]
    perp[MARK_PX] = (float(perp_data['bidPx']) + float(perp_data['askPx'])) / 2.
    perp[INSTRUMENT_NAME] = perp_instrument
    perp[CCY] = f"{coin.upper()}-{baseCcy.upper()}"
    f_rate = get_okx_funding(perp_instrument)
    # add funding payment to spot to get the adjusted perp price
    if is_usdm:
        # starting point: buy 1 unit of spot for SPOT_PX, sell 1 unit of perp, hold for 8hrs
        # funding payment in USD = Funding Rate * size of 1 contract in USD => 1 contract * MARK_PX * Funding rate
        perp[USE_PX] = spot_px + perp[MARK_PX] * f_rate
    else:
        # starting point: buy 1 unit of spot for SPOT_PX, sell SPOT_PX USD worth of perp, hold for 8hrs
        # funding payment in coin = Funding Rate * POSITION_SIZE_USD => funding payment in USD = Funding Rate * POSITION_SIZE_USD/MARK_PX * SPOT_PX
        perp[USE_PX] = spot_px * (1 + spot_px / perp[MARK_PX] * f_rate)
    all_inst.append(perp)
    instruments_info = get_okx_instruments(coin, f"{coin.upper()}-{baseCcy.upper()}")
    for ins in term_instruments:
        fut = {}
        for d in instruments_info:
            inst_id = d['instId']
            if ins == inst_id:
                fut[EXPIRY] = int(d['expTime'])
                break
        term_fut_data = get_okx_ticker(inst_id)['data'][0]
        fut[MARK_PX] = (float(term_fut_data['bidPx']) + float(term_fut_data['askPx'])) / 2.
        fut[INSTRUMENT_NAME] = inst_id
        fut[CCY] = f"{coin.upper()}-{baseCcy.upper()}"
        fut[USE_PX] = fut[MARK_PX]
        all_inst.append(fut)
    return pd.DataFrame.from_records(all_inst, columns = FUTURES_DATA_COLS)
def get_kraken_instruments(symbol: str) -> dict:
    """
    @param symbol, supports [btc (xbt), eth, ltc, bch, xrp]
    @returns instruments (sorted by expiration date)
    """
    # symbol check
    symbol = symbol.lower()
    valid_symbols = ['btc', 'eth', 'ltc', 'bch', 'xrp']
    if symbol not in valid_symbols:
        raise ValueError(f'< {symbol} > is not valid, must be in {valid_symbols}')
    if symbol == 'btc':
        symbol = 'xbt'
    # call instruments API
    resp = requests.get(KRAKEN_BASE_API + "instruments")
    j = resp.json()
    instruments = {}
    for d in j['instruments']:
        # check for inverse perp and for flexible perp
        is_future_inverse = \
            ('underlying' in d) and \
            (d['underlying'] == f'rr_{symbol}usd') and \
            (d['type'] == 'futures_inverse')
        is_future_flexible = \
            ('symbol' in d) and \
            (d['symbol'] == f'PF_{symbol.upper()}USD') and \
            (d['type'] == 'flexible_futures')
        if is_future_inverse or is_future_flexible:
            last_trading_time = d.get('lastTradingTime')
            last_trading_time_ms = None
            # convert string to milliseconds
            if last_trading_time:
                last_trading_time_ms = int(datetime.strptime(last_trading_time,'%Y-%m-%dT%H:%M:%S.%fZ').timestamp() * 1000)
            instruments[d['symbol']] = {
                'type': d['type'],
                'underlying': d.get('underlying'),
                # 'base_ccy': symbol_original.upper(),
                # 'last_trading_time': last_trading_time,
                'expiration_timestamp_ms': last_trading_time_ms,
                'type': d['type'],
                'funding_rate_coefficient': d.get('fundingRateCoefficient'),
            }
    return instruments
def get_kraken_tickers(symbols: str) -> dict:
    """
    @param symbols list of symbols to return prices for
    @return close price of most resolution
    """
    resp = requests.get(KRAKEN_BASE_API, + "/tickers")
    j = resp.json()
    tickers = j['tickers']
    prices = {d['symbol']: (d['bid'] + d['ask'])/2 for d in tickers if d['symbol'].upper() in symbols}
    return prices
def get_last_funding(symbol: str):
    """
    @notice return last funding payment
    @return last funding payment $, last funding timestamp
    """
    resp = requests.get(KRAKEN_BASE_API, + "/tickers")
    j = resp.json()
    #f_rate = [d['fundingRate'] for d in j['tickers'] if d['symbol'].upper() = symbol]
    #return f_rate
""" instruments = get_instruments('btc')
marks = get_tickers(instruments)
instruments_df = pd.DataFrame(instruments).T
# add explicit sort by timestamp
instruments_df.sort_values(by='expiration_timestamp_ms', na_position='first', inplace=True)
# add mark prices
instruments_df['mark_price'] = pd.Series(marks)
# rename index to 'instrument_name'
# (after adding mark prices - so index match)
instruments_df.reset_index(inplace=True)
instruments_df.rename(columns={'index': 'instrument_name'}, inplace=True)
display(instruments_df) """
# flexible futures are multi-asset perp?
# add last perp funding payment
# instrument_name   base_ccy    expiration_timestamp_ms    mark_price      last_funding
# YES               YES        YES                         YES             NO
def calc_term_rates(futs: pd.DataFrame):
    futs['year_frac'] = (futs[EXPIRY] - futs.iloc[0][EXPIRY])/(YEAR_IN_SECONDS * 1000)
    futs['term_yield'] = np.log(futs[USE_PX]/futs.iloc[0][USE_PX])/futs['year_frac']
    futs.loc[0, 'term_yield'] = np.nan
    # special case: for perp, with a constant maturity of 8 hours, we use expected 8hr return
    # interp_8hperp_mark = futs.iloc[0][MARK_PX] + (futs.loc[1, USE_PX] - futs.iloc[0][USE_PX])/3
    # futs.loc[1, 'term_yield'] = np.power(interp_8hperp_mark/futs.iloc[0][USE_PX], 1/futs.loc[1, 'year_frac']) - 1
def calc_fwd_rates(futs: pd.DataFrame):
    shifted = futs.shift(1)
    futs['fwd_yield'] = np.NaN
    futs.loc[1, 'fwd_yield'] = futs.loc[1, 'term_yield']
    futs.loc[2:, 'fwd_yield'] = np.power(np.power(1 + futs['term_yield'], futs['year_frac'])/np.power(1 + shifted['term_yield'], shifted['year_frac']), (futs['year_frac'] - shifted['year_frac'])) - 1

In [11]:
df = get_all_vol()
df = df.loc[:,~df.columns.duplicated()].copy()
df.reindex()
# all coin-margined futs - vs USD
# all usdt-margined futs - vs USDT
(btc_spot, eth_spot, usdt_spot) = get_spot_prices()
# Deribit futs are coin-margined, so /USD
(futs_btc, futs_eth) = preprocess_futures_data(df, btc_spot * usdt_spot, eth_spot * usdt_spot)
# Binance & OKX have both, so need to use the correct spot px
binfuts_btc_cm = get_binance_futures(binance_dapi, 'BTCUSD', btc_spot * usdt_spot, is_usdm = False)
binfuts_btc_um = get_binance_futures(binance_fapi, 'BTCUSDT', btc_spot)
binfuts_eth_cm = get_binance_futures(binance_dapi, 'ETHUSD', eth_spot * usdt_spot, is_usdm = False)
binfuts_eth_um = get_binance_futures(binance_fapi, 'ETHUSDT', eth_spot)
okxfuts_btc_um = get_okx_futures('BTC', 'USDT', btc_spot)
okxfuts_eth_um = get_okx_futures('ETH', 'USDT', eth_spot)
okxfuts_btc_cm = get_okx_futures('BTC', 'USD', btc_spot * usdt_spot, is_usdm = False)
okxfuts_eth_cm = get_okx_futures('ETH', 'USD', eth_spot * usdt_spot, is_usdm = False)
calc_term_rates(futs_btc)
calc_term_rates(binfuts_btc_cm)
calc_term_rates(binfuts_btc_um)
calc_term_rates(okxfuts_btc_cm)
calc_term_rates(okxfuts_btc_um)
calc_term_rates(futs_eth)
calc_term_rates(binfuts_eth_cm)
calc_term_rates(binfuts_eth_um)
calc_term_rates(okxfuts_eth_cm)
calc_term_rates(okxfuts_eth_um)
calc_fwd_rates(futs_btc)
calc_fwd_rates(binfuts_btc_cm)
calc_fwd_rates(binfuts_btc_um)
calc_fwd_rates(okxfuts_btc_cm)
calc_fwd_rates(okxfuts_btc_um)
calc_fwd_rates(futs_eth)
calc_fwd_rates(binfuts_eth_cm)
calc_fwd_rates(binfuts_eth_um)
calc_fwd_rates(okxfuts_eth_cm)
calc_fwd_rates(okxfuts_eth_um)
display(futs_btc)
display(binfuts_btc_cm)
display(binfuts_btc_um)
display(okxfuts_btc_cm)
display(okxfuts_btc_um)
display(futs_eth)
display(binfuts_eth_cm)
display(binfuts_eth_um)
display(okxfuts_eth_cm)
display(okxfuts_eth_um)

ValueError: No objects to concatenate

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = pd.to_datetime(futs_btc[EXPIRY], unit="ms"), y = futs_btc['term_yield'], line_color = 'darkred', mode = 'markers+lines', name = 'Deribit BTC spot yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(futs_eth[EXPIRY], unit="ms"), y = futs_eth['term_yield'], line_color = 'darkblue', mode = 'markers+lines', name = 'Deribit ETH spot yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(okxfuts_btc_cm[EXPIRY], unit="ms"), y = okxfuts_btc_cm['term_yield'], line_color = 'darkviolet', mode = 'markers+lines', name = 'OKX BTC CM spot yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(okxfuts_eth_cm[EXPIRY], unit="ms"), y = okxfuts_eth_cm['term_yield'], line_color = 'darkgreen', mode = 'markers+lines', name = 'OKX ETH CM spot yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(okxfuts_btc_um[EXPIRY], unit="ms"), y = okxfuts_btc_um['term_yield'], line_color = 'darksalmon', mode = 'markers+lines', name = 'OKX BTC UM spot yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(okxfuts_eth_um[EXPIRY], unit="ms"), y = okxfuts_eth_um['term_yield'], line_color = 'darkkhaki', mode = 'markers+lines', name = 'OKX ETH UM spot yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(binfuts_btc_cm[EXPIRY], unit="ms"), y = binfuts_btc_cm['term_yield'], line_color = 'darkgrey', mode = 'markers+lines', name = 'Binance CM BTC spot yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(binfuts_btc_um[EXPIRY], unit="ms"), y = binfuts_btc_um['term_yield'], line_color = 'darkmagenta', mode = 'markers+lines', name = 'Binance UM BTC spot yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(binfuts_eth_cm[EXPIRY], unit="ms"), y = binfuts_eth_cm['term_yield'], line_color = 'darkslateblue', mode = 'markers+lines', name = 'Binance CM ETH spot yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(binfuts_eth_um[EXPIRY], unit="ms"), y = binfuts_eth_um['term_yield'], line_color = 'darkcyan', mode = 'markers+lines', name = 'Binance UM ETH spot yield'))
fig.update_layout(xaxis_title="maturity", yaxis_title="term yield", legend = dict(orientation = 'h', yanchor = 'top', y = -0.15, font=dict(size=8)))
fig.update_layout(autosize=False,width=850, height=500, margin=dict(l=50, r=50, b=120, t=50, pad=4))
fig.update_xaxes(automargin=True)
fig.show()
fig = go.Figure()
fig.add_trace(go.Scatter(x = pd.to_datetime(futs_btc[EXPIRY], unit="ms"), y = futs_btc['fwd_yield'], line_color = 'darkred', mode = 'markers+lines', name = 'Deribit BTC fwd yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(futs_eth[EXPIRY], unit="ms"), y = futs_eth['fwd_yield'], line_color = 'darkblue', mode = 'markers+lines', name = 'Deribit ETH fwd yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(okxfuts_btc_cm[EXPIRY], unit="ms"), y = okxfuts_btc_cm['fwd_yield'], line_color = 'darkviolet', mode = 'markers+lines', name = 'OKX BTC CM fwd yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(okxfuts_eth_cm[EXPIRY], unit="ms"), y = okxfuts_eth_cm['fwd_yield'], line_color = 'darkgreen', mode = 'markers+lines', name = 'OKX ETH CM fwd yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(okxfuts_btc_um[EXPIRY], unit="ms"), y = okxfuts_btc_um['fwd_yield'], line_color = 'darksalmon', mode = 'markers+lines', name = 'OKX BTC UM fwd yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(okxfuts_eth_um[EXPIRY], unit="ms"), y = okxfuts_eth_um['fwd_yield'], line_color = 'darkkhaki', mode = 'markers+lines', name = 'OKX ETH UM fwd yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(binfuts_btc_cm[EXPIRY], unit="ms"), y = binfuts_btc_cm['fwd_yield'], line_color = 'darkgrey', mode = 'markers+lines', name = 'Binance CM BTC fwd yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(binfuts_btc_um[EXPIRY], unit="ms"), y = binfuts_btc_um['fwd_yield'], line_color = 'darkmagenta', mode = 'markers+lines', name = 'Binance UM BTC fwd yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(binfuts_eth_cm[EXPIRY], unit="ms"), y = binfuts_eth_cm['fwd_yield'], line_color = 'darkslateblue', mode = 'markers+lines', name = 'Binance CM ETH fwd yield'))
fig.add_trace(go.Scatter(x = pd.to_datetime(binfuts_eth_um[EXPIRY], unit="ms"), y = binfuts_eth_um['fwd_yield'], line_color = 'darkcyan', mode = 'markers+lines', name = 'Binance UM ETH fwd yield'))
fig.update_layout(xaxis_title="maturity", yaxis_title="fwd yield", legend = dict(orientation = 'h', yanchor = 'top', y = -0.15, font=dict(size=8)))
fig.update_layout(autosize=False,width=850, height=500, margin=dict(l=50, r=50, b=120, t=50, pad=4))
fig.update_xaxes(automargin=True)
fig.show()